In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/loan_data_raw.csv")
df.head()


,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [2]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [3]:
df = df.drop_duplicates()


In [4]:
df = df.dropna(subset=[
    'income',
    'loan_amount',
    'Credit_Score',
    'LTV',
    'dtir1'
])


In [5]:
df = df[
    (df['income'] > 0) &
    (df['loan_amount'] > 0) &
    (df['Credit_Score'] >= 300)
]


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 124089 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         124089 non-null  int64  
 1   year                       124089 non-null  int64  
 2   loan_limit                 121212 non-null  object 
 3   Gender                     124089 non-null  object 
 4   approv_in_adv              123311 non-null  object 
 5   loan_type                  124089 non-null  object 
 6   loan_purpose               123980 non-null  object 
 7   Credit_Worthiness          124089 non-null  object 
 8   open_credit                124089 non-null  object 
 9   business_or_commercial     124089 non-null  object 
 10  loan_amount                124089 non-null  int64  
 11  rate_of_interest           104110 non-null  float64
 12  Interest_rate_spread       104110 non-null  float64
 13  Upfront_charges            101333 

In [7]:
df.to_csv("../data/processed/loan_data_clean.csv", index=False)


In [8]:
df['loan_to_income'] = df['loan_amount'] / df['income']


In [9]:
df['loan_to_income_capped'] = df['loan_to_income'].clip(upper=5)


In [10]:
def credit_score_band(score):
    if score < 580:
        return 'Poor'
    elif score < 670:
        return 'Fair'
    elif score < 740:
        return 'Good'
    else:
        return 'Excellent'

df['credit_score_band'] = df['Credit_Score'].apply(credit_score_band)


In [11]:
df[['loan_to_income_capped', 'credit_score_band']].head()


,loan_to_income_capped,credit_score_band
0,5.0,Excellent
2,5.0,Excellent
3,5.0,Fair
4,5.0,Fair
5,5.0,Excellent


In [12]:
df.to_csv("../data/processed/loan_data_features.csv", index=False)
